<a href="https://colab.research.google.com/github/Ashwin-Prakash-dev/SCNN-Flood-Detection-Model/blob/main/M2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
from google.colab import drive
drive.mount('/content/drive')

DRIVE_BASE = "/content/drive/MyDrive/sen1floods11"
import os
os.makedirs(DRIVE_BASE, exist_ok=True)
print("Drive base:", DRIVE_BASE)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive base: /content/drive/MyDrive/sen1floods11


# Downloading the dataset

In [23]:
!wget -P '/content/drive/MyDrive/Datasets' https://zenodo.org/records/7946594/files/Labels-20230517T191741Z-001.zip?download=1

--2025-08-23 15:51:00--  https://zenodo.org/records/7946594/files/Labels-20230517T191741Z-001.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.43.25, 188.185.45.92, 188.185.48.194, ...
Connecting to zenodo.org (zenodo.org)|188.185.43.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2462219 (2.3M) [application/octet-stream]
Saving to: ‘/content/drive/MyDrive/Datasets/Labels-20230517T191741Z-001.zip?download=1’

Labels-20230517T191 100%[===================>]   2.35M  2.18MB/s    in 1.1s    

2025-08-23 15:51:02 (2.18 MB/s) - ‘/content/drive/MyDrive/Datasets/Labels-20230517T191741Z-001.zip?download=1’ saved [2462219/2462219]



In [24]:
!wget -P '/content/drive/MyDrive/Datasets' https://zenodo.org/records/7946594/files/POST_S1-20230517T191716Z-001.zip?download=1

--2025-08-23 15:51:02--  https://zenodo.org/records/7946594/files/POST_S1-20230517T191716Z-001.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.43.25, 188.185.45.92, 188.185.48.194, ...
Connecting to zenodo.org (zenodo.org)|188.185.43.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 729719788 (696M) [application/octet-stream]
Saving to: ‘/content/drive/MyDrive/Datasets/POST_S1-20230517T191716Z-001.zip?download=1’

POST_S1-20230517T19 100%[===================>] 695.91M  19.7MB/s    in 44s     

2025-08-23 15:51:47 (15.7 MB/s) - ‘/content/drive/MyDrive/Datasets/POST_S1-20230517T191716Z-001.zip?download=1’ saved [729719788/729719788]



In [25]:
!wget -P '/content/drive/MyDrive/Datasets' https://zenodo.org/records/7946594/files/PRE_S1-20230517T191707Z-001.zip?download=1

--2025-08-23 15:51:47--  https://zenodo.org/records/7946594/files/PRE_S1-20230517T191707Z-001.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.48.194, 188.185.45.92, 188.185.43.25, ...
Connecting to zenodo.org (zenodo.org)|188.185.48.194|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1520699949 (1.4G) [application/octet-stream]
Saving to: ‘/content/drive/MyDrive/Datasets/PRE_S1-20230517T191707Z-001.zip?download=1’

PRE_S1-20230517T191 100%[===================>]   1.42G  20.4MB/s    in 76s     

2025-08-23 15:53:04 (19.1 MB/s) - ‘/content/drive/MyDrive/Datasets/PRE_S1-20230517T191707Z-001.zip?download=1’ saved [1520699949/1520699949]



# Extracting from Zip

In [26]:
import zipfile
import os

zip_s1="/content/drive/MyDrive/Datasets/PRE_S1-20230517T191707Z-001.zip?download=1"
zip_s2="/content/drive/MyDrive/Datasets/POST_S1-20230517T191716Z-001.zip?download=1"
zip_labels="/content/drive/MyDrive/Datasets/Labels-20230517T191741Z-001.zip?download=1"

extract_path_s1 = "/content/drive/MyDrive/Datasets/FloodDet/s1"
extract_path_s2 = "/content/drive/MyDrive/Datasets/FloodDet/s2"
extract_path_labels = "/content/drive/MyDrive/Datasets/FloodDet/labels"

with zipfile.ZipFile(zip_s1, 'r') as zip_ref:
    zip_ref.extractall(extract_path_s1)

with zipfile.ZipFile(zip_s2, 'r') as zip_ref:
    zip_ref.extractall(extract_path_s2)

with zipfile.ZipFile(zip_labels, 'r') as zip_ref:
    zip_ref.extractall(extract_path_labels)


# Checking dimensions of TIFF files

In [1]:
!pip install rasterio
import rasterio

file_path = "/content/drive/MyDrive/Datasets/FloodDet/s1/PRE_S1/Ghana_103272_S1_PREHand.tif"

with rasterio.open(file_path) as src:
    print("Width:", src.width)
    print("Height:", src.height)
    print("Number of bands:", src.count)
    print("Shape:", (src.height, src.width, src.count))  # (H, W, Bands)


Width: 512
Height: 512
Number of bands: 2
Shape: (512, 512, 2)


# Checking number of files

In [2]:
import os

print("Number of files in s1:", len(os.listdir("/content/drive/MyDrive/Datasets/FloodDet/s1/PRE_S1")))
print("Number of files in s2:", len(os.listdir("/content/drive/MyDrive/Datasets/FloodDet/s2/POST_S1")))
print("Number of files in labels:", len(os.listdir("/content/drive/MyDrive/Datasets/FloodDet/labels/Labels")))


Number of files in s1: 418
Number of files in s2: 446
Number of files in labels: 446


## Number of files are not equal so that needs to be fixed somehow, otherwise error will rise during model training

In [3]:
import os

folder_s1 = "/content/drive/MyDrive/Datasets/FloodDet/s1/PRE_S1"
folder_s2 = "/content/drive/MyDrive/Datasets/FloodDet/s2/POST_S1"
folder_labels = "/content/drive/MyDrive/Datasets/FloodDet/labels/Labels"

# Get all filenames
s1_files = sorted([f for f in os.listdir(folder_s1) if f.endswith(".tif")])
s2_files = sorted([f for f in os.listdir(folder_s2) if f.endswith(".tif")])
label_files = sorted([f for f in os.listdir(folder_labels) if f.endswith(".tif")])

# Convert to sets
s1_set = set(s1_files)
s2_set = set(s2_files)
label_set = set(label_files)

# Intersection
common_files = s1_set.intersection(s2_set).intersection(label_set)
print("Common files in all three folders:", len(common_files))

# Filter lists
s1_filtered = sorted([f for f in s1_files if f in common_files])
s2_filtered = sorted([f for f in s2_files if f in common_files])
label_filtered = sorted([f for f in label_files if f in common_files])

print("After filtering:")
print("s1:", len(s1_filtered), "s2:", len(s2_filtered), "labels:", len(label_filtered))


Common files in all three folders: 0
After filtering:
s1: 0 s2: 0 labels: 0


In [4]:
print("Sample S1:", s1_files[:5])
print("Sample S2:", s2_files[:5])
print("Sample Labels:", label_files[:5])


Sample S1: ['Ghana_103272_S1_PREHand.tif', 'Ghana_1033830_S1_PREHand.tif', 'Ghana_1078550_S1_PREHand.tif', 'Ghana_1089161_S1_PREHand.tif', 'Ghana_11745_S1_PREHand.tif']
Sample S2: ['Bolivia_103757_S1Hand.tif', 'Bolivia_129334_S1Hand.tif', 'Bolivia_195474_S1Hand.tif', 'Bolivia_23014_S1Hand.tif', 'Bolivia_233925_S1Hand.tif']
Sample Labels: ['Bolivia_103757_LabelHand.tif', 'Bolivia_129334_LabelHand.tif', 'Bolivia_195474_LabelHand.tif', 'Bolivia_23014_LabelHand.tif', 'Bolivia_233925_LabelHand.tif']


## The problem is that post_s1 and labels contain files for Bolivia but pre_s1 does not

In [5]:
import re
import os

folder_s1 = "/content/drive/MyDrive/Datasets/FloodDet/s1/PRE_S1"
folder_s2 = "/content/drive/MyDrive/Datasets/FloodDet/s2/POST_S1"
folder_labels = "/content/drive/MyDrive/Datasets/FloodDet/labels/Labels"

# Get file lists
s1_files = [f for f in os.listdir(folder_s1) if f.endswith(".tif")]
s2_files = [f for f in os.listdir(folder_s2) if f.endswith(".tif")]
label_files = [f for f in os.listdir(folder_labels) if f.endswith(".tif")]

# Function to extract numeric ID
def extract_id(filename):
    match = re.search(r"_(\d+)_", filename)
    return match.group(1) if match else None

# Build maps
s1_map = {extract_id(f): f for f in s1_files if extract_id(f)}
s2_map = {extract_id(f): f for f in s2_files if extract_id(f)}
label_map = {extract_id(f): f for f in label_files if extract_id(f)}

# Intersection by ID
common_ids = set(s1_map.keys()) & set(s2_map.keys()) & set(label_map.keys())
print("Common IDs:", len(common_ids))

# Build aligned lists
s1_filtered = [s1_map[i] for i in sorted(common_ids)]
s2_filtered = [s2_map[i] for i in sorted(common_ids)]
label_filtered = [label_map[i] for i in sorted(common_ids)]

print("Aligned files count:", len(s1_filtered))
print("Example triplet:")
print(s1_filtered[:2], s2_filtered[:2], label_filtered[:2])


Common IDs: 418
Aligned files count: 418
Example triplet:
['USA_1010394_S1_PREHand.tif', 'India_1017769_S1_PREHand.tif'] ['USA_1010394_S1Hand.tif', 'India_1017769_S1Hand.tif'] ['USA_1010394_LabelHand.tif', 'India_1017769_LabelHand.tif']


## This alligned the dataset based on matching IDs

## s1_filtered, s2_filtered, label_filtered are lists that contain the filenames of the alligned files as strings, so for processing reading these lists we give (only) the alligned files

# Converting to NumPy array

In [6]:
import numpy as np
import rasterio
import os
from tqdm import tqdm

def load_tif_as_array(filepath):
    with rasterio.open(filepath) as src:
        img = src.read()  # shape: (bands, H, W)
        img = np.transpose(img, (1, 2, 0))  # to (H, W, bands)
    return img

# Directories
s1_dir = "/content/drive/MyDrive/Datasets/FloodDet/s1/PRE_S1"
s2_dir = "/content/drive/MyDrive/Datasets/FloodDet/s2/POST_S1"
label_dir = "/content/drive/MyDrive/Datasets/FloodDet/labels/Labels"

# Initialize lists
s1_data = []
s2_data = []
label_data = []

# Keep track of files that cause errors
error_files = []

for s1_file, s2_file, label_file in tqdm(zip(s1_filtered, s2_filtered, label_filtered), total=len(s1_filtered)):
    s1_fp = os.path.join(s1_dir, s1_file)
    s2_fp = os.path.join(s2_dir, s2_file)
    label_fp = os.path.join(label_dir, label_file)

    try:
        # Load arrays
        s1_img = load_tif_as_array(s1_fp)  # (512, 512, bands)
        s2_img = load_tif_as_array(s2_fp)
        lbl_img = load_tif_as_array(label_fp)

        # Normalize S1 and S2
        s1_img = s1_img.astype(np.float32)
        s2_img = s2_img.astype(np.float32)

        # Min-max normalization per image
        s1_img = (s1_img - s1_img.min()) / (s1_img.max() - s1_img.min() + 1e-6)
        s2_img = (s2_img - s2_img.min()) / (s2_img.max() - s2_img.min() + 1e-6)

        # Labels (already 0/1 typically)
        lbl_img = lbl_img.astype(np.int32)

        # Append
        s1_data.append(s1_img)
        s2_data.append(s2_img)
        label_data.append(lbl_img)
    except rasterio.errors.RasterioIOError:
        print(f"Skipping triplet due to RasterioIOError: {s1_file}, {s2_file}, {label_file}")
        error_files.append((s1_file, s2_file, label_file))


# Convert to NumPy arrays
s1_data = np.array(s1_data)       # shape: (N, 512, 512, bands)
s2_data = np.array(s2_data)       # shape: (N, 512, 512, bands)
label_data = np.array(label_data) # shape: (N, 512, 512, 1)

print(f"Skipped {len(error_files)} triplets due to errors.")

100%|██████████| 418/418 [01:16<00:00,  5.44it/s]


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (418,) + inhomogeneous part.

In [7]:
for i, (s1_img, s2_img, lbl_img) in enumerate(zip(s1_data, s2_data, label_data)):
    if s1_img.shape != (512, 512, s1_img.shape[2]) or s2_img.shape != (512, 512, s2_img.shape[2]) or lbl_img.shape[:2] != (512, 512):
        print(f"Index {i}:")
        print(f"  S1 shape: {s1_img.shape}")
        print(f"  S2 shape: {s2_img.shape}")
        print(f"  Label shape: {lbl_img.shape}")


Index 37:
  S1 shape: (512, 374, 2)
  S2 shape: (512, 512, 2)
  Label shape: (512, 512, 1)
Index 52:
  S1 shape: (512, 94, 2)
  S2 shape: (512, 512, 2)
  Label shape: (512, 512, 1)
Index 53:
  S1 shape: (512, 69, 2)
  S2 shape: (512, 512, 2)
  Label shape: (512, 512, 1)
Index 76:
  S1 shape: (512, 368, 2)
  S2 shape: (512, 512, 2)
  Label shape: (512, 512, 1)
Index 94:
  S1 shape: (439, 512, 2)
  S2 shape: (512, 512, 2)
  Label shape: (512, 512, 1)
Index 101:
  S1 shape: (512, 246, 2)
  S2 shape: (512, 512, 2)
  Label shape: (512, 512, 1)
Index 113:
  S1 shape: (512, 463, 2)
  S2 shape: (512, 512, 2)
  Label shape: (512, 512, 1)
Index 116:
  S1 shape: (512, 281, 2)
  S2 shape: (512, 512, 2)
  Label shape: (512, 512, 1)
Index 120:
  S1 shape: (512, 335, 2)
  S2 shape: (512, 512, 2)
  Label shape: (512, 512, 1)
Index 177:
  S1 shape: (512, 485, 2)
  S2 shape: (512, 512, 2)
  Label shape: (512, 512, 1)
Index 202:
  S1 shape: (512, 354, 2)
  S2 shape: (512, 512, 2)
  Label shape: (512, 512

## resizing s1 files

In [8]:
import cv2

TARGET_SIZE = (512, 512)

s1_dir = "/content/drive/MyDrive/Datasets/FloodDet/s1/PRE_S1"
s2_dir = "/content/drive/MyDrive/Datasets/FloodDet/s2/POST_S1"
labels_dir = "/content/drive/MyDrive/Datasets/FloodDet/labels/Labels"

def resize_image(img, target_size, interpolation=cv2.INTER_LINEAR):
    return cv2.resize(img, target_size, interpolation=interpolation)

s1_data = []
s2_data = []
label_data = []

for s1_file, s2_file, label_file in tqdm(zip(s1_filtered, s2_filtered, label_filtered), total=len(s1_filtered)):
    s1_fp = os.path.join(s1_dir, s1_file)
    s2_fp = os.path.join(s2_dir, s2_file)
    label_fp = os.path.join(labels_dir, label_file)

    s1_img = load_tif_as_array(s1_fp)
    s2_img = load_tif_as_array(s2_fp)
    lbl_img = load_tif_as_array(label_fp)

    # Normalize
    s1_img = (s1_img - s1_img.min()) / (s1_img.max() - s1_img.min() + 1e-6)
    s2_img = (s2_img - s2_img.min()) / (s2_img.max() - s2_img.min() + 1e-6)

    # Resize S1 only (since it's inconsistent)
    if s1_img.shape[:2] != (512, 512):
        s1_img = resize_image(s1_img, TARGET_SIZE)

    # Resize labels (just in case) using nearest neighbor
    if lbl_img.shape[:2] != (512, 512):
        lbl_img = resize_image(lbl_img, TARGET_SIZE, interpolation=cv2.INTER_NEAREST)

    s1_data.append(s1_img)
    s2_data.append(s2_img)
    label_data.append(lbl_img)

s1_data = np.array(s1_data)       # (N, 512, 512, 2)
s2_data = np.array(s2_data)       # (N, 512, 512, 2)
label_data = np.array(label_data) # (N, 512, 512, 1)


100%|██████████| 418/418 [00:57<00:00,  7.25it/s]


In [9]:

for i, (s1_img, s2_img, lbl_img) in enumerate(zip(s1_data, s2_data, label_data)):
    if s1_img.shape != (512, 512, 2):
        print(f"Index {i}: S1 shape mismatch: {s1_img.shape}")
    if s2_img.shape != (512, 512, 2):
        print(f"Index {i}: S2 shape mismatch: {s2_img.shape}")
    if lbl_img.shape != (512, 512) and lbl_img.shape != (512, 512, 1):
        print(f"Index {i}: Label shape mismatch: {lbl_img.shape}")

print("Verification complete.")


Verification complete.


## Cropping Size


In [10]:
import tensorflow as tf
import rasterio
import numpy as np
import os
from sklearn.model_selection import train_test_split

s1_files = s1_filtered
s2_files = s2_filtered
label_files = label_filtered

(s1_tr, s1_va,
 s2_tr, s2_va,
 y_tr,  y_va) = train_test_split(s1_files, s2_files, label_files, test_size=0.2, random_state=42)

def read_tif(path):
    with rasterio.open(path.numpy().decode()) as src:
        arr = src.read()  # shape: (bands, H, W)
    arr = np.transpose(arr, (1, 2, 0))  # (H, W, bands)
    arr = arr.astype("float32") / 10000.0  # normalize
    return arr

def load_triplet(s1_path, s2_path, label_path):
    s1 = tf.py_function(read_tif, [s1_path], tf.float32)
    s2 = tf.py_function(read_tif, [s2_path], tf.float32)
    lbl = tf.py_function(read_tif, [label_path], tf.float32)
    s1.set_shape([None, None, None])
    s2.set_shape([None, None, None])
    lbl.set_shape([None, None, None])
    return (s1, s2), lbl

CROP_SIZE = 256
def random_crop(s1, s2, lbl):
    stacked = tf.concat([s1, s2, lbl], axis=-1)
    cropped = tf.image.random_crop(stacked, size=[CROP_SIZE, CROP_SIZE, tf.shape(stacked)[-1]])
    s1_crop = cropped[:, :, :tf.shape(s1)[-1]]
    s2_crop = cropped[:, :, tf.shape(s1)[-1]:tf.shape(s1)[-1]+tf.shape(s2)[-1]]
    lbl_crop = cropped[:, :, -1:]
    return (s1_crop, s2_crop), lbl_crop

# Build tf.data pipeline
def make_dataset(s1_list, s2_list, lbl_list, batch_size=4, shuffle=True):
    ds = tf.data.Dataset.from_tensor_slices((s1_list, s2_list, lbl_list))
    if shuffle:
        ds = ds.shuffle(len(s1_list))
    ds = ds.map(lambda s1, s2, l: load_triplet(s1, s2, l), num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.map(lambda x, y: random_crop(x[0], x[1], y), num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

BATCH = 2
train_ds = make_dataset(s1_tr, s2_tr, y_tr, batch_size=BATCH, shuffle=True)
val_ds   = make_dataset(s1_va, s2_va, y_va, batch_size=BATCH, shuffle=False)



# Model Training

In [11]:
import tensorflow as tf
from tensorflow.keras import layers, Model

def dice_coef(y_true, y_pred, smooth=1.0):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    y_pred = tf.clip_by_value(y_pred, 1e-7, 1-1e-7)
    intersection = tf.reduce_sum(y_true * y_pred, axis=[1,2,3])
    denom = tf.reduce_sum(y_true, axis=[1,2,3]) + tf.reduce_sum(y_pred, axis=[1,2,3])
    dice = (2. * intersection + smooth) / (denom + smooth)
    return tf.reduce_mean(dice)

def iou_coef(y_true, y_pred, smooth=1.0):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    y_pred = tf.clip_by_value(y_pred, 1e-7, 1-1e-7)
    intersection = tf.reduce_sum(y_true * y_pred, axis=[1,2,3])
    union = tf.reduce_sum(y_true + y_pred, axis=[1,2,3]) - intersection
    iou = (intersection + smooth) / (union + smooth)
    return tf.reduce_mean(iou)

def bce_dice_loss(y_true, y_pred):
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    return tf.reduce_mean(bce) + (1.0 - dice_coef(y_true, y_pred))

def conv_block(x, filters):
    x = layers.Conv2D(filters, 3, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Conv2D(filters, 3, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    return x

def encoder_stem(inp, base=64):
    c1 = conv_block(inp, base)
    p1 = layers.MaxPooling2D()(c1)
    c2 = conv_block(p1, base*2)
    p2 = layers.MaxPooling2D()(c2)
    c3 = conv_block(p2, base*4)
    p3 = layers.MaxPooling2D()(c3)
    c4 = conv_block(p3, base*8)
    p4 = layers.MaxPooling2D()(c4)
    bottleneck = conv_block(p4, base*16)
    return (c1, c2, c3, c4), bottleneck

def decoder_fused(bott, s1_skips, s2_skips, base=32, num_classes=1):
    x = layers.Concatenate()([bott[0], bott[1]])
    for f, (s1, s2) in zip([base*8, base*4, base*2, base], zip(reversed(s1_skips), reversed(s2_skips))):
        x = layers.Conv2DTranspose(f, 2, strides=2, padding="same")(x)
        x = layers.Concatenate()([x, s1, s2])
        x = conv_block(x, f)
    out = layers.Conv2D(num_classes, 1, activation="sigmoid")(x)
    return out

def siamese_unet(input_shape=(256,256,2), base=32, share_encoders=True, num_classes=1):
    inp_s1 = layers.Input(shape=input_shape, name="S1_pre")
    inp_s2 = layers.Input(shape=input_shape, name="S2_post")

    if share_encoders:
        s_in = layers.Input(shape=input_shape)
        s_skips, s_bott = encoder_stem(s_in, base=base)
        encoder_model = Model(s_in, [s_skips, s_bott], name="shared_encoder")

        s1_skips, s1_bott = encoder_model(inp_s1)
        s2_skips, s2_bott = encoder_model(inp_s2)
    else:
        s1_skips, s1_bott = encoder_stem(inp_s1, base=base)
        s2_skips, s2_bott = encoder_stem(inp_s2, base=base)

    out = decoder_fused((s1_bott, s2_bott), s1_skips, s2_skips, base=base, num_classes=num_classes)
    model = Model([inp_s1, inp_s2], out, name="SiameseUNet")
    return model


In [ ]:

s1 = s1_data.astype("float32")
s2 = s2_data.astype("float32")
y  = label_data.astype("float32")

from sklearn.model_selection import train_test_split
(s1_tr, s1_va,
 s2_tr, s2_va,
 y_tr,  y_va) = train_test_split(s1, s2, y, test_size=0.2, random_state=42)

# tf.data pipelines
BATCH = 2
train_ds = tf.data.Dataset.from_tensor_slices(((s1_tr, s2_tr), y_tr)).shuffle(256).batch(BATCH).prefetch(tf.data.AUTOTUNE)
val_ds   = tf.data.Dataset.from_tensor_slices(((s1_va, s2_va), y_va)).batch(BATCH).prefetch(tf.data.AUTOTUNE)

model = siamese_unet(input_shape=(256,256,2), base=32, share_encoders=True, num_classes=1)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss=bce_dice_loss,
              metrics=[dice_coef, iou_coef, "accuracy"])
model.summary()

# Train
callbacks = [
    tf.keras.callbacks.ModelCheckpoint("siamese_unet_best.h5", save_best_only=True, monitor="val_dice_coef", mode="max"),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3),
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=8, restore_best_weights=True)
]
history = model.fit(train_ds, validation_data=val_ds, epochs=30, callbacks=callbacks)
